In [ ]:
# import libraries
import tensorflow as tf
import numpy as np

In [ ]:

# Define the batch size
batch_size = 20

# Define a function for data preprocessing
def preprocess_image(image, label):
    # Normalize pixel values to the range [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Preprocess and create TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(preprocess_image)
train_dataset = train_dataset.shuffle(buffer_size=60000).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(preprocess_image)
test_dataset = test_dataset.batch(batch_size)

# Create data iterators for training and testing
#train_data = iter(train_dataset)
#test_data = iter(test_dataset)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # Flatten the input
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
## Define the NN architecture
class Net(tf.keras.Model):
    def __init__(self):
        super(Net, self).__init__()

        self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
        self.fc1 = tf.keras.layers.Dense(512, activation='relu')
        self.fc2 = tf.keras.layers.Dense(256, activation='relu')
        self.fc3 = tf.keras.layers.Dense(128, activation='relu')
        self.fc4 = tf.keras.layers.Dense(64, activation='relu')
        self.fc5 = tf.keras.layers.Dense(10, activation='softmax')
        self.dropout = tf.keras.layers.Dropout(0.5)


    def call(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.dropout(x)
        x = self.fc5(x)
        return x

# initialize the NN
model = Net()
print(model)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()  # Changed optimizer to Adam

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])


In [ ]:
# Number of epochs to train the model
n_epochs = 10

for epoch in range(n_epochs):
    train_loss = 0.0

    for data, target in train_dataset:
        with tf.GradientTape() as tape:
            output = model(data, training=True)
            loss = loss_fn(target, output)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss += loss.numpy()

    train_loss = train_loss / len(train_dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))


Epoch: 1 	Training Loss: 0.651917
Epoch: 2 	Training Loss: 0.328588
Epoch: 3 	Training Loss: 0.279013
Epoch: 4 	Training Loss: 0.239703
Epoch: 5 	Training Loss: 0.228043
Epoch: 6 	Training Loss: 0.204213
Epoch: 7 	Training Loss: 0.193412
Epoch: 8 	Training Loss: 0.188731
Epoch: 9 	Training Loss: 0.178332
Epoch: 10 	Training Loss: 0.170848


In [ ]:
test_loss = 0.0
class_correct = [0] * 10
class_total = [0] * 10

for data, target in test_dataset:
    output = model(data, training=False)
    loss = loss_fn(target, output)
    test_loss += loss.numpy()

    pred = tf.argmax(output, axis=1)
    target = tf.cast(target, tf.int64)
    correct = np.squeeze(pred == target)

    for i in range(batch_size):
        label = target[i].numpy()
        class_correct[label] += correct[i]
        class_total[label] += 1

test_loss = test_loss / len(test_dataset)

print('Test Loss: {:.6f}\n'.format(test_loss))

Test Loss: 0.112703



In [ ]:

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (str(i)))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100 * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)
))

Test Accuracy of     0: 99% (973/980)
Test Accuracy of     1: 98% (1123/1135)
Test Accuracy of     2: 97% (1008/1032)
Test Accuracy of     3: 96% (977/1010)
Test Accuracy of     4: 97% (962/982)
Test Accuracy of     5: 97% (872/892)
Test Accuracy of     6: 97% (932/958)
Test Accuracy of     7: 97% (1005/1028)
Test Accuracy of     8: 94% (924/974)
Test Accuracy of     9: 95% (964/1009)

Test Accuracy (Overall): 97% (9740/10000)


In [ ]:
## Define the NN architecture
class Net(tf.keras.Model):
    def __init__(self):
        super(Net, self).__init__()

        self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
        self.fc1 = tf.keras.layers.Dense(512, activation='relu')
        self.fc2 = tf.keras.layers.Dense(256, activation='relu')
        self.fc3 = tf.keras.layers.Dense(128, activation='relu')
        self.fc4 = tf.keras.layers.Dense(64, activation='relu')
        self.fc5 = tf.keras.layers.Dense(10, activation='softmax')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(0.5)


    def call(self, x):
        x = self.flatten(x)
        x = self.fc1(self.bn1(x))
        x = self.dropout(x)
        x = self.fc2(self.bn2(x))
        x = self.dropout(x)
        x = self.fc3(self.bn3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.dropout(x)
        x = self.fc5(x)
        return x

# initialize the NN
model = Net()
print(model)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()  # Changed optimizer to Adam

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])


In [ ]:
# Number of epochs to train the model
n_epochs = 10

for epoch in range(n_epochs):
    train_loss = 0.0

    for data, target in train_dataset:
        with tf.GradientTape() as tape:
            output = model(data, training=True)
            loss = loss_fn(target, output)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss += loss.numpy()

    train_loss = train_loss / len(train_dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))


Epoch: 1 	Training Loss: 0.958155
Epoch: 2 	Training Loss: 0.557449
Epoch: 3 	Training Loss: 0.474393
Epoch: 4 	Training Loss: 0.424521
Epoch: 5 	Training Loss: 0.398124
Epoch: 6 	Training Loss: 0.379559
Epoch: 7 	Training Loss: 0.364446
Epoch: 8 	Training Loss: 0.346973
Epoch: 9 	Training Loss: 0.343348
Epoch: 10 	Training Loss: 0.338310


In [ ]:
test_loss = 0.0
class_correct = [0] * 10
class_total = [0] * 10

for data, target in test_dataset:
    output = model(data, training=False)
    loss = loss_fn(target, output)
    test_loss += loss.numpy()

    pred = tf.argmax(output, axis=1)
    target = tf.cast(target, tf.int64)
    correct = np.squeeze(pred == target)

    for i in range(batch_size):
        label = target[i].numpy()
        class_correct[label] += correct[i]
        class_total[label] += 1

test_loss = test_loss / len(test_dataset)

print('Test Loss: {:.6f}\n'.format(test_loss))

Test Loss: 0.207289



In [ ]:

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (str(i)))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100 * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)
))

Test Accuracy of     0: 98% (967/980)
Test Accuracy of     1: 98% (1123/1135)
Test Accuracy of     2: 96% (996/1032)
Test Accuracy of     3: 96% (974/1010)
Test Accuracy of     4: 96% (949/982)
Test Accuracy of     5: 96% (858/892)
Test Accuracy of     6: 96% (923/958)
Test Accuracy of     7: 96% (988/1028)
Test Accuracy of     8: 93% (914/974)
Test Accuracy of     9: 94% (956/1009)

Test Accuracy (Overall): 96% (9648/10000)
